In [1]:
!pip install -q spacy scikit-learn
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Sample FAQs
faqs = [
    {"question": "What is your return policy?", "answer": "You can return any product within 30 days of purchase."},
    {"question": "How do I track my order?", "answer": "Go to your orders page and click on 'Track Order'."},
    {"question": "Do you offer international shipping?", "answer": "Yes, we ship to over 50 countries worldwide."},
    {"question": "How can I contact customer support?", "answer": "You can email us at support@example.com."},
    {"question": "What payment methods do you accept?", "answer": "We accept Visa, MasterCard, PayPal, and Apple Pay."}
]

# Preprocess function
def preprocess(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Preprocess questions and build TF-IDF matrix
questions = [preprocess(faq["question"]) for faq in faqs]
vectorizer = TfidfVectorizer()
faq_matrix = vectorizer.fit_transform(questions)


In [3]:
def get_answer(user_input):
    user_processed = preprocess(user_input)
    user_vec = vectorizer.transform([user_processed])
    similarities = cosine_similarity(user_vec, faq_matrix)
    best_match = similarities.argmax()
    confidence = similarities[0][best_match]

    if confidence > 0.3:
        return faqs[best_match]["answer"]
    else:
        return "Sorry, I couldn't understand your question. Please try rephrasing it."


In [4]:
def chatbot():
    print("🤖 FAQ Chatbot (type 'exit' to quit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Bot: Goodbye!")
            break
        response = get_answer(user_input)
        print("Bot:", response)

chatbot()


🤖 FAQ Chatbot (type 'exit' to quit)
You: what about Orders
Bot: Go to your orders page and click on 'Track Order'.
You: how can i know about my orders
Bot: Go to your orders page and click on 'Track Order'.
You: exit
Bot: Goodbye!
